## Imports

In [54]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import urllib3
import re
from textwrap import shorten
import os
import geopandas as gpd

## Read-in

In [55]:
df = pd.read_csv('Sites list (2).xlsx - Sheet1.csv')

In [56]:
df['full_address'] = df['Address'] + ' Manhattan, NY'

## Google Maps API Geolocater Setup

In [57]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [ ]:
df['geo_address'] = df['full_address']
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## Correction Section

In [ ]:
df.columns

In [ ]:
df = df.dropna(subset='Address')

In [ ]:
df = df.fillna('Not provided')

## Add colors to DF

In [ ]:
df['Color'] = ''
df.loc[df['Type'] == 'Public', 'Color'] = 'Purple'
df.loc[df['Type'] == 'Private', 'Color'] = 'Green'
df.loc[df['Type'] == 'Mixed', 'Color'] = 'Orange'

## Separate DF into private, public and mixed sub-DFs

In [ ]:
df_Public = df.loc[df['Type'] == 'Public'] ## Purple
df_Private = df.loc[df['Type'] == 'Private'] ## Green
df_Mixed = df.loc[df['Type'] == 'Mixed'] ## Orange

In [ ]:
df.Type.value_counts()

## HTML popup formatter

In [ ]:
def popup_html(row):
    i = row
    Address = df['Address'].iloc[i]
    Type = df['Type'].iloc[i]
    Owner = df['Owner'].iloc[i]
    Current_Use = df['Current Use'].iloc[i]
    Zoning = df['Zoning'].iloc[i]
    Commercial_Overlay = df['Commercial Overlay'].iloc[i]
    Max_Units = df['Max Units'].iloc[i]
    Proposed_Units = df['Proposed Units'].iloc[i]
    Affordability = df['Affordability'].iloc[i]
    Prez_comments = df['MBPO Comments'].iloc[i]
    
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>Type: </strong>{}'''.format(Type) + '''<br>
    <strong>Owner: </strong>{}'''.format(Owner) + '''<br>
    <strong>Current Use: </strong>{}'''.format(Current_Use) + '''<br>
    <strong>Zoning: </strong>{}'''.format(Zoning) + '''<br>
    <strong>Commercial Overlay: </strong>{}'''.format(Commercial_Overlay) + '''<br>
    <strong>Max Units: </strong>{}'''.format(Max_Units) + '''<br>
    <strong>Proposed Units: </strong>{}'''.format(Proposed_Units) + '''<br>
    <strong>Affordability: </strong>{}'''.format(Affordability) + '''<br>
    <strong>Comments: </strong>{}'''.format(Prez_comments) + '''
    </html>
    '''
    return html


## Map Maker

In [ ]:
import folium
import branca

### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=10.5,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"Manhattan Development Opportunities")

m.get_root().html.add_child(folium.Element(title_html))

### Create Public Pins ###
for i in range(0,len(df_Public)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True), min_width=400, max_width=400)
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup, icon=folium.Icon(color='purple')).add_to(m)
    
### Create Private Pins ###
for i in range(0,len(df_Private)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True), min_width=400, max_width=400)
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup, icon=folium.Icon(color='green')).add_to(m)
    
### Create Mixed Pins ###
for i in range(0,len(df_Mixed)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True), min_width=400, max_width=400)
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup, icon=folium.Icon(color='orange')).add_to(m)
    

lgd_txt = '<span style="color: {col};">{txt}</span>'

### Create legend ###
for i, r in df.iterrows():
    fg = folium.FeatureGroup(name=lgd_txt.format(txt=r['Type'] + ' ' + '= ' + r['Color'],col=r['Color']))
    m.add_child(fg)
    
folium.TileLayer('Open Street Map',control=False).add_to(m)
    
folium.map.LayerControl('topleft', collapsed=False).add_to(m)

m

In [ ]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [ ]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [ ]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

## Data Check

In [ ]:
df.Owner.value_counts().head(60)

In [ ]:
df.loc[df['Owner'] == 'Shermans Creek Dev. Co.']